In [1]:
!pip install sklearn-genetic


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sn
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt
%matplotlib inline

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-4rqiajrn because the default path (/home/pranay/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [3]:
df = pd.read_csv("final.csv")
print(type(df))
new_df=df.to_numpy()
print(type(new_df))
X=new_df[:,[2,3,4,5,6]]
print(X)
y=new_df[:,1]
print(y)

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
[[3 2 4 4 2]
 [2 2 4 2 1]
 [2 2 1 1 2]
 ...
 [2 3 1 1 1]
 [2 1 4 2 1]
 [1 2 4 1 2]]
[0 0 0 ... 0 0 0]


In [12]:
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree

clf = DecisionTreeClassifier(max_depth=5, random_state=1234)
model = clf.fit(X, y)

In [13]:
text_representation = tree.export_text(clf)
print(text_representation)

|--- feature_0 <= 2.50
|   |--- feature_0 <= 1.50
|   |   |--- feature_3 <= 1.50
|   |   |   |--- feature_0 <= 0.50
|   |   |   |   |--- feature_4 <= 0.50
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- feature_4 >  0.50
|   |   |   |   |   |--- class: 0
|   |   |   |--- feature_0 >  0.50
|   |   |   |   |--- feature_4 <= 2.50
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- feature_4 >  2.50
|   |   |   |   |   |--- class: 0
|   |   |--- feature_3 >  1.50
|   |   |   |--- feature_1 <= 0.50
|   |   |   |   |--- feature_3 <= 3.50
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- feature_3 >  3.50
|   |   |   |   |   |--- class: 1
|   |   |   |--- feature_1 >  0.50
|   |   |   |   |--- feature_1 <= 3.50
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- feature_1 >  3.50
|   |   |   |   |   |--- class: 0
|   |--- feature_0 >  1.50
|   |   |--- feature_3 <= 0.50
|   |   |   |--- feature_2 <= 0.50
|   |   |   |   |--- feature_1 <= 2.50
|   |   |   |   |   |--- class: 

In [14]:
import pandas as pd
feature_df = pd.read_csv("final.csv")
keys=list(feature_df.keys())
new_key = []
req = [2,3,4,5,6]
for i in req:
    new_key.append(keys[i])
target_key=keys[1]
print(new_key)
print(target_key)

['VehicleAge', 'VehOdo', 'ARCP', 'VehBCost', 'WarrantyCost']
IsBadBuy


In [15]:
text_representation = tree.export_text(clf, feature_names=new_key)
print(text_representation)

|--- VehicleAge <= 2.50
|   |--- VehicleAge <= 1.50
|   |   |--- VehBCost <= 1.50
|   |   |   |--- VehicleAge <= 0.50
|   |   |   |   |--- WarrantyCost <= 0.50
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- WarrantyCost >  0.50
|   |   |   |   |   |--- class: 0
|   |   |   |--- VehicleAge >  0.50
|   |   |   |   |--- WarrantyCost <= 2.50
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- WarrantyCost >  2.50
|   |   |   |   |   |--- class: 0
|   |   |--- VehBCost >  1.50
|   |   |   |--- VehOdo <= 0.50
|   |   |   |   |--- VehBCost <= 3.50
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- VehBCost >  3.50
|   |   |   |   |   |--- class: 1
|   |   |   |--- VehOdo >  0.50
|   |   |   |   |--- VehOdo <= 3.50
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- VehOdo >  3.50
|   |   |   |   |   |--- class: 0
|   |--- VehicleAge >  1.50
|   |   |--- VehBCost <= 0.50
|   |   |   |--- ARCP <= 0.50
|   |   |   |   |--- VehOdo <= 2.50
|   |   |   |   |   |--- class: 1
|   | 

In [16]:
from sklearn.tree import _tree

def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
            p1 += [f"({name} <= {np.round(threshold, 3)})"]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [f"({name} > {np.round(threshold, 3)})"]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = "if "
        
        for p in path[:-1]:
            if rule != "if ":
                rule += " and "
            rule += str(p)
        rule += " then "
        if class_names is None:
            rule += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            rule += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
        rule += f" | based on {path[-1][1]:,} samples"
        rules += [rule]
        
    return rules

In [17]:
rules = get_rules(clf, new_key, target_key)
for r in rules:
    print(r)
    print()

if (VehicleAge <= 2.5) and (VehicleAge > 1.5) and (VehBCost > 0.5) and (WarrantyCost <= 3.5) and (ARCP > 0.5) then class: I (proba: 89.19%) | based on 4,691 samples

if (VehicleAge <= 2.5) and (VehicleAge <= 1.5) and (VehBCost > 1.5) and (VehOdo > 0.5) and (VehOdo <= 3.5) then class: I (proba: 95.21%) | based on 3,173 samples

if (VehicleAge <= 2.5) and (VehicleAge <= 1.5) and (VehBCost <= 1.5) and (VehicleAge > 0.5) and (WarrantyCost <= 2.5) then class: I (proba: 90.13%) | based on 1,936 samples

if (VehicleAge > 2.5) and (VehBCost > 0.5) and (VehBCost <= 2.5) and (ARCP > 1.5) and (VehicleAge <= 3.5) then class: I (proba: 80.31%) | based on 1,046 samples

if (VehicleAge <= 2.5) and (VehicleAge > 1.5) and (VehBCost <= 0.5) and (ARCP > 0.5) and (ARCP > 1.5) then class: I (proba: 80.5%) | based on 646 samples

if (VehicleAge <= 2.5) and (VehicleAge > 1.5) and (VehBCost <= 0.5) and (ARCP > 0.5) and (ARCP <= 1.5) then class: I (proba: 84.57%) | based on 525 samples

if (VehicleAge > 2.5) a